<a href="https://colab.research.google.com/github/CM747/Book-Listing-app/blob/master/Assignment2_17110037_Chandan_Maji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import random
import re
import math
import operator
import string

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [105]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **DATA PREPROCESSING**

In [0]:
fp = open("/content/drive/My Drive/speeches.txt", 'r', encoding="utf-8-sig", errors='ignore')
data = fp.read()
data = data.lower()

# DATA Preprocessing
data = data.replace("\n", "")
data = data.replace("...", ". ")
data = re.sub(r"SPEECH [0-9]","", data)
data = re.sub(r"[:;]",".",data)

# Sentence Tokenizer
sent_tokenize_list = sent_tokenize(data)

lines = list()
for i in range(len(sent_tokenize_list)):
  if(len(sent_tokenize_list[i])>1):
    lines.append("<s> " + sent_tokenize_list[i][:-1] + " </s>")

# Train Test Split
random.shuffle(lines)
cut = int(0.8*len(lines))
train = lines[:cut]
test = lines[cut:]

# CLASSICAL APPROACH

In [299]:
vocab_dict = {}
vocab_len = 0
for i in range(len(train)):
  tokens = train[i].split(" ")
  for j in tokens:
    if(j in vocab_dict):
      vocab_dict[j] += 1
    else:
      vocab_dict[j] = 1
      vocab_len += 1
  
tokens = sum(vocab_dict.values())
vocab_len

10314

In [300]:
tokens

153331

In [0]:
# function to extract n_grams and their frequencies
def ngramCounts(lines, n_gram):
  counts = dict()
  for i in range(len(lines)):
    tokens = lines[i].split(" ")
    for j in range(0,len(tokens)-n_gram+1,1):
      k = tuple(tokens[j: j+n_gram])
      if k in counts:
        counts[k] += 1
      else:
        counts[k] = 1
  return counts

## Maximum Likelihood Estimation

In [0]:
# Functions to get Maximum Likelihood Estimate Probabilities for unigrams, bigrams, trigrams and quadgrams
count_unigram = ngramCounts(lines, 1)
count_bigram = ngramCounts(lines, 2)
count_trigram = ngramCounts(lines, 3)
count_quadgram = ngramCounts(lines, 4)

In [0]:
def unigramMLE(word_list):
  if(tuple(word_list) in count_unigram):
    return (count_unigram[tuple(word_list)])/(tokens)
  else:
    return 0

def bigramMLE(word_list):
  if(tuple(word_list) in count_bigram):
    return (count_bigram[tuple(word_list)])/(count_unigram[tuple(word_list[:-1])])
  else:
    return 0

def trigramMLE(word_list):
  if(tuple(word_list) in count_trigram):
    return (count_trigram[tuple(word_list)])/(count_bigram[tuple(word_list[:-1])])
  else:
    return 0


def quadgramMLE(word_list):
  if(tuple(word_list) in count_quadgram):
    return (count_quadgram[tuple(word_list)])/(count_trigram[tuple(word_list[:-1])])
  else:
    return 0

In [317]:
print(unigramMLE(["the"]))
print(bigramMLE(["it", "is"]))
print(trigramMLE(["agree","it","should"]))
print(quadgramMLE(["agree", "it", "should", "be"]))

0.037663616620252915
0.045366795366795366
1.0
1.0


## Generator using Classical Method

In [0]:
#Classical generator to generate new words using a n_gram model
def MLE_Generator(n_gram, initial_sequence):
  sentence = ["<s>"]
  if(n_gram==1):
    for i in range(20):
      max_prob = 0
      max_prob_list = list()
      for j in vocab_dict.keys():
        k = (unigramMLE([j]))
        if(k>max_prob):
          max_prob = k
          max_prob_list = [j]
        elif(k==max_prob):
          max_prob_list.append(j)
      samples = np.random.multinomial(20,[max_prob]*len(max_prob_list),size=1)
      index, value = max(enumerate(samples), key = operator.itemgetter(1))
      sentence.append(max_prob_list[index])
  else:
    sentence.extend(initial_sequence)
    i = len(initial_sequence)
    while(sentence[-1]!="</s>" and i<20):
      max_prob = 0
      max_prob_list = list()
      for j in vocab:
        word_list = sentence[-n_gram+1:]
        word_list.append(j)
        if(n_gram==2):
          k = bigramMLE(word_list)
        elif(n_gram==3):
          k = trigramMLE(word_list)
        elif(n_gram==4):
          k = quadgramMLE(word_list)
        if(k>max_prob):
          max_prob = k
          max_prob_list = [j]
        elif(k==max_prob):
          max_prob_list.append(j)
      samples = np.random.multinomial(20,[max_prob]*len(max_prob_list),size=1)
      index, value = max(enumerate(samples), key = operator.itemgetter(1))
      sentence.append(max_prob_list[index])
      i+=1
  
  return sentence

## Evaluation of Classical Method

In [0]:
# function to calculate sentence probability of given n_gram model
def Probability(sentence, n_gram):
  prob = 0
  sentence = sentence.split(" ")
  if(n_gram==1):
    fun = unigramMLE
    sentence = sentence[1:-1]
  elif(n_gram==2):
    fun = bigramMLE
  elif(n_gram==3):
    fun = trigramMLE
  else:
    fun = quadgramMLE
  
  for i in range(len(sentence)-n_gram+1):
    w = sentence[i:i+n_gram]
    prob += math.log(fun(w),2)
  
  return math.pow(2,prob)


# function to calculate perplexity of all sentences given n_gram model
def Perplexity(sentences, n_gram):
  tot_len = 0
  prob = 0

  for i in sentences:
    try:
      prob -= math.log(Probability(i, n_gram),2)
    except:
      prob -= 0
    sentence = i.split(" ")
    tot_len += (len(sentence)-n_gram-1)
  
  return math.pow(2, prob/tot_len)

### Perplexity Calculations

In [320]:
# Calculating perplexity of all sentences in test dataset

print("unigram perplexity: ",Perplexity(test, 1))
print("bigram perplexity: ",Perplexity(test, 2))
print("trigram perplexity: ",Perplexity(test, 3))
print("quadgram perplexity: ",Perplexity(test, 4))

unigram perplexity:  589.1865432242371
bigram perplexity:  58.26548504814822
trigram perplexity:  6.053377064057492
quadgram perplexity:  1.881737040439814


### Random Text Generation

In [322]:
# GENERATING RANDOM TEXT:
# UNIGRAMS
print("UNIGRAM: ", end = " ")
print(" ".join(MLE_Generator(1,[])))
# BIGRAM
print("BIGRAM: ", end = " ")
print(" ".join(MLE_Generator(2,["it", "is"])))
# TRIGRAM
print("TRIGRAM: ", end = " ")
print(" ".join(MLE_Generator(3,["there", "is", "something"])))
# QUADGRAM
print("QUADGRAM1: ", end = " ")
print(" ".join(MLE_Generator(4,["but", "there", "is", "no"])))
# ANOTHER QuADGRAM
print("QUADGRAM2: ", end = " ")
print(" ".join(MLE_Generator(4,["it", "is", "to", "be"])))

UNIGRAM:  <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s>
BIGRAM:  <s> it is a lot of the world </s>
TRIGRAM:  <s> there is something that i can tell you that </s>
QUADGRAM1:  <s> but there is no leadership at the top </s>
QUADGRAM2:  <s> it is to be treated a little bit </s>


Unigram Keeps on printing the word with maximum probability, as unigram does not keep an account of the context words.
Bigram does better than unigram but still prints out sentences that make no sense.
Trigram and Quadgram does much better than the previous two, printing sentences that make more sense and are readable.

# NEURAL APPROACH

In [0]:
# NEURAL APPROACH
sequences = list()
n_gram = 3
for i in range(len(train)):
  tokens = train[i].split(" ")
  if(len(tokens)>=n_gram):
    for k in range(0,len(tokens)-n_gram+1):
      sequences.append(tokens[k:k+n_gram])

In [191]:
sequences[:10]

[['<s>', "they're", 'walking'],
 ["they're", 'walking', '</s>'],
 ['<s>', 'i', 'killed'],
 ['i', 'killed', 'the'],
 ['killed', 'the', 'applause'],
 ['the', 'applause', '</s>'],
 ['<s>', 'it’s', 'a'],
 ['it’s', 'a', 'crooked'],
 ['a', 'crooked', 'outfit.look,'],
 ['crooked', 'outfit.look,', 'club']]

In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences_intEncoded = tokenizer.texts_to_sequences(sequences)

In [193]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

10199


In [238]:
sequences_intEncoded[:10]

array([[   4,  366, 1327],
       [ 366, 1327,    5],
       [   4,    6,  511],
       [   6,  511,    1],
       [ 511,    1, 1908],
       [   1, 1908,    5],
       [   4,   26,    7],
       [  26,    7,  951],
       [   7,  951, 4507],
       [ 951, 4507, 1093]])

In [195]:
sequences_intEncoded = np.array(sequences_intEncoded)
np.shape(sequences)

(131432, 3)

In [0]:
X = sequences_intEncoded[:,:-1]
y = sequences_intEncoded[:,-1]

In [0]:
y = to_categorical(y, num_classes=vocab_size)

## VANILLA RNN MODEL

In [203]:
# VANILLA RNN MODEL
from keras.layers import SimpleRNN

model_rnn = Sequential()
model_rnn.add(Embedding(vocab_size, 50, input_length=n_gram-1))
model_rnn.add(SimpleRNN(25, return_sequences=True))
model_rnn.add(SimpleRNN(25))
model_rnn.add(Dense(50, activation='relu'))
model_rnn.add(Dense(vocab_size, activation='softmax'))

model_rnn.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 2, 50)             509950    
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 2, 25)             1900      
_________________________________________________________________
simple_rnn_10 (SimpleRNN)    (None, 25)                1275      
_________________________________________________________________
dense_17 (Dense)             (None, 50)                1300      
_________________________________________________________________
dense_18 (Dense)             (None, 10199)             520149    
Total params: 1,034,574
Trainable params: 1,034,574
Non-trainable params: 0
_________________________________________________________________


In [204]:
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn = model_rnn.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
131432/131432 [==============================] - 98s 747us/step - loss: 6.2811 - acc: 0.1017
Epoch 2/10
131432/131432 [==============================] - 96s 728us/step - loss: 5.4971 - acc: 0.1518
Epoch 3/10
131432/131432 [==============================] - 95s 723us/step - loss: 5.1230 - acc: 0.1786
Epoch 4/10
131432/131432 [==============================] - 96s 728us/step - loss: 4.8673 - acc: 0.1978
Epoch 5/10
131432/131432 [==============================] - 95s 721us/step - loss: 4.6687 - acc: 0.2126
Epoch 6/10
131432/131432 [==============================] - 96s 727us/step - loss: 4.4980 - acc: 0.2248
Epoch 7/10
131432/131432 [==============================] - 97s 735us/step - loss: 4.3434 - acc: 0.2361
Epoch 8/10
131432/131432 [==============================] - 95s 726us/step - loss: 4.1995 - acc: 0.2470
Epoch 9/10
131432/131432 [==============================] - 92s 697us/step - loss: 4.0643 - acc: 0.2583
Epoch 10/10
131432/131432 [==============================] - 89s

### Generating Random Text using Vanilla RNN

In [265]:
for i in range(5):
  result = list()
  text = test[random.randint(0,len(test))].split(" ")
  f = 0
  if(len(text)>=n_gram):
    result = text[:n_gram-1]
    l = n_gram
    while(result[-1]!="</s>"):
      encoded = tokenizer.texts_to_sequences([result[-n_gram+1:]])
      encoded = np.array(encoded)
      yhat = model_rnn.predict_classes(encoded, verbose=0)
      out_word = ''
      for word, index in tokenizer.word_index.items():
        if index == yhat:
          out_word = word
          break
      result.append(out_word)
      l+=1
    print(" ".join(result))

<s> let’s do it </s>
<s> if you know what </s>
<s> when you have to be a great job </s>
<s> we’re going to be a great job </s>
<s> now, i have a lot of money </s>


## LSTM MODEL

In [200]:
# LSTM MODEL
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_size, 50, input_length=n_gram-1))
model_lstm.add(LSTM(25, return_sequences=True))
model_lstm.add(LSTM(25))
model_lstm.add(Dense(50, activation='relu'))
model_lstm.add(Dense(vocab_size, activation='softmax'))

model_lstm.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 2, 50)             509950    
_________________________________________________________________
lstm_7 (LSTM)                (None, 2, 25)             7600      
_________________________________________________________________
lstm_8 (LSTM)                (None, 25)                5100      
_________________________________________________________________
dense_15 (Dense)             (None, 50)                1300      
_________________________________________________________________
dense_16 (Dense)             (None, 10199)             520149    
Total params: 1,044,099
Trainable params: 1,044,099
Non-trainable params: 0
_________________________________________________________________


In [206]:
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_lstm = model_lstm.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
131432/131432 [==============================] - 103s 784us/step - loss: 5.9425 - acc: 0.1161
Epoch 2/10
131432/131432 [==============================] - 99s 752us/step - loss: 5.5952 - acc: 0.1413
Epoch 3/10
131432/131432 [==============================] - 99s 752us/step - loss: 5.3575 - acc: 0.1612
Epoch 4/10
131432/131432 [==============================] - 100s 760us/step - loss: 5.1744 - acc: 0.1742
Epoch 5/10
131432/131432 [==============================] - 99s 757us/step - loss: 5.0386 - acc: 0.1840
Epoch 6/10
131432/131432 [==============================] - 100s 757us/step - loss: 4.9216 - acc: 0.1910
Epoch 7/10
131432/131432 [==============================] - 97s 740us/step - loss: 4.8120 - acc: 0.1990
Epoch 8/10
131432/131432 [==============================] - 97s 738us/step - loss: 4.7075 - acc: 0.2075
Epoch 9/10
131432/131432 [==============================] - 96s 732us/step - loss: 4.6058 - acc: 0.2140
Epoch 10/10
131432/131432 [==============================] - 

### Generating Random Text Using LSTM

In [270]:
for i in range(5):
  result = list()
  text = test[random.randint(0,len(test))].split(" ")
  f = 0
  if(len(text)>=n_gram):
    result = text[:n_gram-1]
    l = n_gram
    while(result[-1]!="</s>"):
      encoded = tokenizer.texts_to_sequences([result[-n_gram+1:]])
      encoded = np.array(encoded)
      yhat = model_lstm.predict_classes(encoded, verbose=0)
      out_word = ''
      for word, index in tokenizer.word_index.items():
        if index == yhat:
          out_word = word
          break
      result.append(out_word)
      l+=1
    print(" ".join(result))

<s> they have to be a lot of people </s>
<s> he was going to be a lot of people </s>
<s> it’s a lot of people </s>
<s> you’re going to be a lot of people </s>
<s> sorry </s>


### Does Neural Approach perform better than Classical Approach. Why or why not?
Yes.
In neural approach the network learns how to use words, while in classical approach it is just a function of probability.
In neural approach, context is remembered for a longer duration than in classical methods, where context in remembered only upto certain words (0 for unigram, 1 for bigram, 2 for trigram and 3 for quadgram). Also LSTM model remembers more than vanilla RNN, due to vanishing gradients in Vanilla RNNs.
Neural approach also takes into account the similarity between words and hence can output a more sensible sentence than classical methods